# 特徴量設計検討ノート

このノートブックでは、競馬予想に使用する特徴量の検討・設計を行います。

## 設計方針
**レースレベル（1着賞金）× 着順** をベースにした特徴量を構築する

## 目次
1. HTMLから取得可能なデータの確認
2. 特徴量設計
3. HTMLパーサーのプロトタイプ
4. 特徴量計算のプロトタイプ

---
## 1. 環境設定とデータ確認

In [ ]:
import os
import sys
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルートへのパス設定
PROJECT_ROOT = Path('../../')
DATA_DIR = PROJECT_ROOT / 'data' / 'raceHTML'

print(f"データディレクトリ: {DATA_DIR.resolve()}")
print(f"年別フォルダ: {sorted([d.name for d in DATA_DIR.glob('*') if d.is_dir()])}")

---
## 2. 特徴量設計

### 2.1 レースレベル分類（1着賞金ベース）

| レベル | 賞金範囲（万円） | レース例 |
|--------|-----------------|----------|
| S | 10,000以上 | G1 |
| A | 5,000-9,999 | G2 |
| B | 3,000-4,999 | G3, リステッド |
| C | 1,500-2,999 | オープン, 3勝クラス |
| D | 750-1,499 | 2勝クラス, 1勝クラス |
| E | 750未満 | 新馬, 未勝利 |

In [ ]:
def classify_race_level(prize_money):
    """
    1着賞金からレースレベルを分類
    
    Args:
        prize_money: 1着賞金（万円）
    Returns:
        level: レースレベル（S/A/B/C/D/E）
        level_score: 数値スコア（予測に使用）
    """
    if prize_money >= 10000:
        return 'S', 6
    elif prize_money >= 5000:
        return 'A', 5
    elif prize_money >= 3000:
        return 'B', 4
    elif prize_money >= 1500:
        return 'C', 3
    elif prize_money >= 750:
        return 'D', 2
    else:
        return 'E', 1

# テスト
test_prizes = [30000, 7000, 4000, 2000, 1000, 550]
for p in test_prizes:
    level, score = classify_race_level(p)
    print(f"{p}万円 → レベル{level} (スコア:{score})")

### 2.2 主要特徴量

#### 過去成績ベース

| 特徴量 | 説明 | 計算式 |
|--------|------|--------|
| `weighted_avg_finish` | レベル加重平均着順 | Σ(着順 × レベルスコア) / Σレベルスコア |
| `best_finish_high_level` | 高レベル（A以上）最高着順 | min(着順) where level >= A |
| `level_win_count` | レベル別勝利数 | count where 着順 == 1 per level |
| `level_place_rate` | レベル別複勝率 | count(着順<=3) / count per level |

#### 今回レースとの比較

| 特徴量 | 説明 |
|--------|------|
| `level_gap` | 今回レベル - 過去平均レベル（正=格上挑戦）|
| `same_level_count` | 同レベル出走経験数 |
| `same_level_best` | 同レベル最高着順 |

---
## 3. HTMLパーサーのプロトタイプ

In [ ]:
def parse_race_html(html_path):
    """
    レースHTMLをパースして構造化データを抽出する
    
    Returns:
        dict: race_info（レース情報）, horses（出走馬リスト）
    """
    with open(html_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
    
    race_id = Path(html_path).stem
    race_info = {'race_id': race_id}
    
    # レース詳細
    race_data = soup.find('dl', class_='racedata')
    if race_data:
        race_info['race_name'] = race_data.find('h1').text.strip() if race_data.find('h1') else ''
        
        span = race_data.find('span')
        if span:
            info_text = span.text.strip()
            # 距離・コース
            match = re.search(r'(芝|ダート)(右|左)?(外)?(\d+)m', info_text)
            if match:
                race_info['surface'] = match.group(1)
                race_info['direction'] = match.group(2) or ''
                race_info['course_type'] = match.group(3) or ''
                race_info['distance'] = int(match.group(4))
            
            # 天候・馬場
            weather_match = re.search(r'天候 : (\S+)', info_text)
            if weather_match:
                race_info['weather'] = weather_match.group(1)
            
            condition_match = re.search(r'(芝|ダート) : (\S+)', info_text)
            if condition_match:
                race_info['track_condition'] = condition_match.group(2)
    
    # 日付
    date_elem = soup.find('p', class_='smalltxt')
    if date_elem:
        date_match = re.search(r'(\d{4})年(\d{2})月(\d{2})日', date_elem.text)
        if date_match:
            race_info['date'] = f"{date_match.group(1)}-{date_match.group(2)}-{date_match.group(3)}"
    
    # 出走馬情報
    horses = []
    result_table = soup.find('table', class_='race_table_01')
    if result_table:
        for row in result_table.find_all('tr')[1:]:
            cells = row.find_all('td')
            if len(cells) < 14:
                continue
            
            horse = {'race_id': race_id}
            
            # 着順
            finish = cells[0].text.strip()
            horse['finish_position'] = int(finish) if finish.isdigit() else None
            
            # 枠番・馬番
            horse['gate_number'] = int(cells[1].text.strip()) if cells[1].text.strip().isdigit() else None
            horse['horse_number'] = int(cells[2].text.strip()) if cells[2].text.strip().isdigit() else None
            
            # 馬名・馬ID
            horse_link = cells[3].find('a')
            if horse_link:
                horse['horse_name'] = horse_link.text.strip()
                href = horse_link.get('href', '')
                match = re.search(r'/horse/(\d+)/', href)
                if match:
                    horse['horse_id'] = match.group(1)
            
            # 性齢
            sex_age = cells[4].text.strip()
            if sex_age:
                horse['sex'] = sex_age[0]
                horse['age'] = int(sex_age[1:]) if len(sex_age) > 1 and sex_age[1:].isdigit() else None
            
            # 斤量
            try:
                horse['weight'] = float(cells[5].text.strip())
            except:
                horse['weight'] = None
            
            # 騎手
            jockey_link = cells[6].find('a')
            if jockey_link:
                horse['jockey_name'] = jockey_link.text.strip()
                href = jockey_link.get('href', '')
                match = re.search(r'/jockey/result/recent/(\d+)/', href)
                if match:
                    horse['jockey_id'] = match.group(1)
            
            # タイム
            horse['time'] = cells[7].text.strip()
            
            # 着差
            horse['margin'] = cells[8].text.strip()
            
            # 通過順（diary_snap_cutの中にある）
            # 上がり3F
            # オッズ・人気・馬体重は後続のセルから
            
            # 賞金（最後のセル）
            prize_text = cells[-1].text.strip()
            try:
                horse['prize_money'] = float(prize_text)
            except:
                horse['prize_money'] = 0.0
            
            horses.append(horse)
    
    # 1着賞金をレース情報に追加
    if horses:
        first_place = [h for h in horses if h.get('finish_position') == 1]
        if first_place:
            race_info['first_prize'] = first_place[0].get('prize_money', 0)
            level, score = classify_race_level(race_info['first_prize'])
            race_info['race_level'] = level
            race_info['level_score'] = score
    
    return {'race_info': race_info, 'horses': horses}

In [ ]:
# テスト: サンプルHTMLをパース
sample_html_path = DATA_DIR / '2024' / '202401010101.html'
result = parse_race_html(sample_html_path)

print("=== レース情報 ===")
for k, v in result['race_info'].items():
    print(f"  {k}: {v}")

print(f"\n=== 出走馬 ({len(result['horses'])}頭) ===")
for h in result['horses'][:3]:
    print(f"  {h.get('finish_position')}着: {h.get('horse_name')} (ID:{h.get('horse_id')}) 賞金:{h.get('prize_money')}万")

---
## 4. 特徴量計算のプロトタイプ

In [ ]:
def calculate_horse_features(horse_id, history_df, current_race_level_score):
    """
    馬の過去成績から特徴量を計算
    
    Args:
        horse_id: 馬ID
        history_df: 過去レース結果のDataFrame
        current_race_level_score: 今回レースのレベルスコア
    
    Returns:
        dict: 特徴量辞書
    """
    # 該当馬の過去成績を抽出
    horse_history = history_df[history_df['horse_id'] == horse_id].copy()
    
    if len(horse_history) == 0:
        return {'horse_id': horse_id, 'race_count': 0}
    
    features = {
        'horse_id': horse_id,
        'race_count': len(horse_history),
    }
    
    # 着順が記録されているレースのみ
    finished = horse_history[horse_history['finish_position'].notna()]
    
    if len(finished) > 0:
        # レベル加重平均着順
        weighted_sum = (finished['finish_position'] * finished['level_score']).sum()
        weight_total = finished['level_score'].sum()
        features['weighted_avg_finish'] = weighted_sum / weight_total if weight_total > 0 else None
        
        # 平均着順
        features['avg_finish'] = finished['finish_position'].mean()
        
        # 勝率・複勝率
        features['win_rate'] = (finished['finish_position'] == 1).sum() / len(finished)
        features['place_rate'] = (finished['finish_position'] <= 3).sum() / len(finished)
        
        # 高レベル（レベルスコア5以上=A以上）での成績
        high_level = finished[finished['level_score'] >= 5]
        if len(high_level) > 0:
            features['high_level_count'] = len(high_level)
            features['high_level_best'] = high_level['finish_position'].min()
            features['high_level_place_rate'] = (high_level['finish_position'] <= 3).sum() / len(high_level)
        else:
            features['high_level_count'] = 0
            features['high_level_best'] = None
            features['high_level_place_rate'] = None
        
        # 今回レースレベルとの差
        avg_level = finished['level_score'].mean()
        features['level_gap'] = current_race_level_score - avg_level
        
        # 同レベルでの成績
        same_level = finished[finished['level_score'] == current_race_level_score]
        if len(same_level) > 0:
            features['same_level_count'] = len(same_level)
            features['same_level_best'] = same_level['finish_position'].min()
            features['same_level_place_rate'] = (same_level['finish_position'] <= 3).sum() / len(same_level)
        else:
            features['same_level_count'] = 0
            features['same_level_best'] = None
            features['same_level_place_rate'] = None
    
    return features

print("特徴量計算関数を定義しました")

---
## 5. 次のステップ

1. **S02_feature_engineering.ipynb（Sagemaker）**
   - 全HTMLをパースしてDataFrame化
   - 特徴量計算の一括実行
   - `data/features/` に保存

2. **S03_model_training.ipynb（Sagemaker）**
   - 特徴量データセットを使ったモデル学習

3. **L01_image_parser.ipynb（ローカル）**
   - 出馬表画像から horse_id を取得
   - 特徴量DBと照合して予想